In [1]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import ssl

uri = "(<MongoDBString>n
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'), tls=True,
                             tlsAllowInvalidCertificates=True)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


Pinged your deployment. You successfully connected to MongoDB!


In [2]:
db = client.get_database('housing')


In [3]:
propertyinfo = db.get_collection('propertyinfo')


In [4]:
realtorlisting = db.get_collection('realtorlisting')

In [5]:
realtor = db.get_collection('realtor')


In [6]:
import pandas as pd

In [7]:

# Define the aggregation pipeline
pipeline = [
    {
        '$lookup': {
            'from': 'realtorlisting',
            'localField': 'MLSNum',
            'foreignField': 'MLSNum',
            'as': 'listings'
        }
    },
    {
        '$unwind': '$listings'
    },
    {
        '$lookup': {
            'from': 'realtor',
            'localField': 'listings.AgentID',
            'foreignField': 'AgentID',
            'as': 'realtor'
        }
    },
    {
        '$unwind': '$realtor'
    },
    {
        '$project': {
            '_id': 0,
            'MLSNum': 1,
            'Status': 1,
            'StatusChangeTimestamp': 1,
            'PropType': 1,
            'ListPrice': 1,
            'ClosePrice': 1,
            'Address': 1,
            'City': 1,
            'Acres': 1,
            'SqFtTotal': 1,
            'SqFtEstHeatedAboveGrade': 1,
            'StyleorRentType': 1,
            'RoomsTotal': 1,
            'BedsTotal': 1,
            'Bathrooms': 1,
            'Garage/Park': 1,
            'YearBuilt': 1,
            'DaysonMarket': 1,
            'AgentID': '$realtor.AgentID',
            'Name': '$realtor.Name',
            'PhoneNumber': '$realtor.PhoneNumber',
            'Email': '$realtor.Email'
        }
    }
]

# Aggregate the data
result = list(propertyinfo.aggregate(pipeline))

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result)

# Export the DataFrame to a CSV file
df.to_csv('merged_data.csv', index=False)


In [8]:
client.close()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read in the CSV file with your df
df = pd.read_csv('MySQLMergedData.csv')

# Check the number of rows and columns
print("Number of rows and columns:", df.shape)

# Display the first 5 rows of the df
print("First 5 rows of the df:")
print(df.head())

# Check for missing values
print("Number of missing values in each column:")
print(df.isnull().sum())

# Summary statistics for numeric variables
print("Summary statistics for numeric variables:")
print(df.describe())

# Distribution of each numeric variable
for col in df.select_dtypes(include=['int64', 'float64']):
    plt.figure()
    df[col].plot(kind='hist')
    plt.title(col)

# Bar chart of each categorical variable
for col in df.select_dtypes(include=['object']):
    plt.figure()
    df[col].value_counts().plot(kind='bar')
    plt.title(col)

# Correlation matrix of numeric variables
corr = df.corr()
plt.figure()
plt.matshow(corr)
plt.xticks(range(len(corr.columns)), corr.columns)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.colorbar()

# Scatterplot of two numeric variables
plt.figure()
plt.scatter(df['ListPrice'], df['ClosePrice'])
plt.xlabel('ListPrice')
plt.ylabel('ClosePrice')
plt.title('ListPrice vs ClosePrice')

plt.show()
